# Abusive Content Recognizer

## Setup

In [1]:
import keras.layers as layers
import keras
import tensorflow as tf
import pandas
import numpy as np

2023-09-22 23:36:48.633737: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-09-22 23:36:50.988583: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


## Load Data

Dataset from the Kaggle jigsaw content moderation contest as well as 1000 items from surge.ai's publicly available dataset

In [2]:
train_dataset = pandas.read_csv("train.csv")
train_dataset

,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,0000997932d777bf,Explanation\nWhy the edits made under my usern...,0,0,0,0,0,0
1,000103f0d9cfb60f,D'aww! He matches this background colour I'm s...,0,0,0,0,0,0
2,000113f07ec002fd,"Hey man, I'm really not trying to edit war. It...",0,0,0,0,0,0
3,0001b41b1c6bb37e,"""\nMore\nI can't make any real suggestions on ...",0,0,0,0,0,0
4,0001d958c54c6e35,"You, sir, are my hero. Any chance you remember...",0,0,0,0,0,0
...,...,...,...,...,...,...,...,...
159566,ffe987279560d7ff,""":::::And for the second time of asking, when ...",0,0,0,0,0,0
159567,ffea4adeee384e90,You should be ashamed of yourself \n\nThat is ...,0,0,0,0,0,0
159568,ffee36eab5c267c9,"Spitzer \n\nUmm, theres no actual article for ...",0,0,0,0,0,0
159569,fff125370e4aaaf3,And it looks like it was actually you who put ...,0,0,0,0,0,0


In [3]:
train_dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 159571 entries, 0 to 159570
Data columns (total 8 columns):
 #   Column         Non-Null Count   Dtype 
---  ------         --------------   ----- 
 0   id             159571 non-null  object
 1   comment_text   159571 non-null  object
 2   toxic          159571 non-null  int64 
 3   severe_toxic   159571 non-null  int64 
 4   obscene        159571 non-null  int64 
 5   threat         159571 non-null  int64 
 6   insult         159571 non-null  int64 
 7   identity_hate  159571 non-null  int64 
dtypes: int64(6), object(2)
memory usage: 9.7+ MB


In [4]:
train_dataset["toxic"].min()

0

In [5]:
import tensorflow_hub as hub
import tensorflow_text as text
text_input = tf.keras.layers.Input(shape=(), dtype=tf.string)
preprocessor = hub.KerasLayer(
    "https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3")
encoder = hub.KerasLayer(
    "https://tfhub.dev/tensorflow/bert_en_uncased_L-24_H-1024_A-16/4",
    trainable=True)

2023-09-22 23:36:55.744124: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:996] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-09-22 23:36:56.180804: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:996] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-09-22 23:36:56.181239: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:996] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysf

## Preprocess

In [6]:
def preprocess(data: pandas.DataFrame):
    x = data["comment_text"].values
    y = np.zeros(len(data), dtype=np.float32)
    for i in range(len(data)):
        y[i] = data["toxic"][i]
        y[i] = y[i] or data["severe_toxic"][i]
        y[i] = y[i] or data["obscene"][i]
        y[i] = y[i] or data["threat"][i]
        y[i] = y[i] or data["insult"][i]
        y[i] = y[i] or data["identity_hate"][i]

    return x, y

def preprocessOther():
    data = pandas.read_csv("labels.csv")
    x = data["text"].values
    y = data["is_toxic"].replace({"Toxic":1, "Not Toxic":0}, inplace=False)
    return x, y
        

In [7]:
x, y = preprocessOther()

In [8]:
X, Y = preprocess(train_dataset)

In [9]:
temp = np.column_stack([X, Y])
temp.shape

(159571, 2)

In [10]:
Y[np.nonzero(Y)]

array([1., 1., 1., ..., 1., 1., 1.], dtype=float32)

In [11]:
zero_locats = np.zeros(Y.shape[0] - np.count_nonzero(Y), dtype=np.int32)
idx = 0
for i in range(Y.shape[0]):
    if Y[i] != 1:
        zero_locats[idx] = i
        idx += 1
zero_locats

array([     0,      1,      2, ..., 159568, 159569, 159570], dtype=int32)

In [12]:
num_positives = np.count_nonzero(Y)
real_x = np.concatenate([X[np.nonzero(Y)], X[zero_locats][:num_positives]])
real_y = np.concatenate([Y[np.nonzero(Y)], Y[zero_locats][:num_positives]])


In [13]:
real_x.shape

(32450,)

In [14]:
# final_x = np.concatenate([real_x, x])
# final_y = np.concatenate([real_y, y])
final_x = np.concatenate([real_x, x])
final_y = np.concatenate([real_y, y])
final_x.shape

(33450,)

## Model

In [15]:
def create_model():
    inputs = layers.Input(shape=(), dtype=tf.string)
    x = preprocessor(inputs)
    outputs = encoder(x)["pooled_output"]
    
    # feedforward
    x2 = layers.Dropout(0.1)(outputs)
    x2 = layers.Dense(1024, activation='gelu')(outputs)
    x2 = layers.Dropout(0.1)(x2)
    x3 = layers.Dense(1, activation='sigmoid')(x2)
    return keras.Model(inputs=inputs, outputs = x3)
model = create_model()
model.summary()

2023-09-22 23:37:31.355006: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'inputs' with dtype string and shape [?]
	 [[{{node inputs}}]]
2023-09-22 23:37:31.376256: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder' with dtype string and shape [?]
	 [[{{node Placeholder}}]]
2023-09-22 23:37:31.877711: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'inputs' with dtype int32 and shape [?,128]
	 [[{{node inputs}}]]
2023-09-22 2

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_2 (InputLayer)           [(None,)]            0           []                               
                                                                                                  
 keras_layer (KerasLayer)       {'input_type_ids':   0           ['input_2[0][0]']                
                                (None, 128),                                                      
                                 'input_mask': (Non                                               
                                e, 128),                                                          
                                 'input_word_ids':                                                
                                (None, 128)}                                                  

In [16]:
model.compile(
    tf.keras.optimizers.AdamW(1e-5),
    loss=tf.losses.BinaryCrossentropy(),
    metrics=[tf.keras.metrics.BinaryCrossentropy(), tf.keras.metrics.BinaryAccuracy()],
    # weighted_metrics=[tf.keras.metrics.BinaryCrossentropy(), tf.keras.metrics.BinaryAccuracy()]
)

## Train

In [17]:
model.fit(final_x, final_y, batch_size=8, epochs=8, validation_split=0.1, shuffle=True)

Epoch 1/8


2023-09-22 23:37:32.617239: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'inputs' with dtype int32 and shape [?,128]
	 [[{{node inputs}}]]
2023-09-22 23:37:32.617357: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'inputs_2' with dtype int32 and shape [?,128]
	 [[{{node inputs_2}}]]
2023-09-22 23:37:43.280375: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/bert_encoder/StatefulPartitionedCall_grad/bert_encoder/StatefulPart

3764/3764 [==============================] - 1951s 496ms/step - loss: 0.1955 - binary_crossentropy: 0.1955 - binary_accuracy: 0.9234 - val_loss: 0.2410 - val_binary_crossentropy: 0.2410 - val_binary_accuracy: 0.9025
Epoch 2/8
3764/3764 [==============================] - 1852s 492ms/step - loss: 0.1175 - binary_crossentropy: 0.1175 - binary_accuracy: 0.9549 - val_loss: 0.2521 - val_binary_crossentropy: 0.2521 - val_binary_accuracy: 0.9121
Epoch 3/8
3764/3764 [==============================] - 1850s 492ms/step - loss: 0.0708 - binary_crossentropy: 0.0708 - binary_accuracy: 0.9744 - val_loss: 0.3625 - val_binary_crossentropy: 0.3625 - val_binary_accuracy: 0.8765
Epoch 4/8
3764/3764 [==============================] - 1853s 492ms/step - loss: 0.0368 - binary_crossentropy: 0.0368 - binary_accuracy: 0.9884 - val_loss: 0.3648 - val_binary_crossentropy: 0.3648 - val_binary_accuracy: 0.9079
Epoch 5/8
3764/3764 [==============================] - 1851s 492ms/step - loss: 0.0248 - binary_crossentro

## Save

In [18]:
model.save("recognizer")

2023-09-23 03:46:02.702675: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'inputs' with dtype string and shape [?]
	 [[{{node inputs}}]]
2023-09-23 03:46:02.729680: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'input_2' with dtype string and shape [?]
	 [[{{node input_2}}]]
2023-09-23 03:46:02.882970: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'inputs' with dtype int32 and shape [?,128]
	 [[{{node inputs}}]]
2023-09-23 03:46:02.

INFO:tensorflow:Unsupported signature for serialization: ((IndexedSlicesSpec(TensorShape([None, 1024]), tf.float32, tf.int32, tf.int32, TensorShape([None])), <tensorflow.python.framework.func_graph.UnknownArgument object at 0x7f65aab3fd90>, 140074725951792), {}).


INFO:tensorflow:Unsupported signature for serialization: ((IndexedSlicesSpec(TensorShape([None, 1024]), tf.float32, tf.int32, tf.int32, TensorShape([None])), <tensorflow.python.framework.func_graph.UnknownArgument object at 0x7f65aab3fd90>, 140074725951792), {}).


INFO:tensorflow:Unsupported signature for serialization: ((TensorSpec(shape=(512, 1024), dtype=tf.float32, name='gradient'), <tensorflow.python.framework.func_graph.UnknownArgument object at 0x7f62d5575890>, 140074725951600), {}).


INFO:tensorflow:Unsupported signature for serialization: ((TensorSpec(shape=(512, 1024), dtype=tf.float32, name='gradient'), <tensorflow.python.framework.func_graph.UnknownArgument object at 0x7f62d5575890>, 140074725951600), {}).


...

INFO:tensorflow:Unsupported signature for serialization: ((TensorSpec(shape=(1024, 1024), dtype=tf.float32, name='gradient'), <tensorflow.python.framework.func_graph.UnknownArgument object at 0x7f62cfa88910>, 140074727476400), {}).


INFO:tensorflow:Unsupported signature for serialization: ((TensorSpec(shape=(1024,), dtype=tf.float32, name='gradient'), <tensorflow.python.framework.func_graph.UnknownArgument object at 0x7f62cfa8d090>, 140074727475520), {}).


INFO:tensorflow:Unsupported signature for serialization: ((TensorSpec(shape=(1024,), dtype=tf.float32, name='gradient'), <tensorflow.python.framework.func_graph.UnknownArgument object at 0x7f62cfa8d090>, 140074727475520), {}).
2023-09-23 03:46:11.873818: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'serving_default_input_2' with dtype string and shape [?]
	 [[{{node serving_default_input_2}}]]


INFO:tensorflow:Assets written to: recognizer/assets


INFO:tensorflow:Assets written to: recognizer/assets


## Evaluate

In [19]:
def evaluate(text:str):
    preds = model(tf.constant([text])).numpy()[0]
    return preds
    

In [20]:
evaluate("Hi there")

array([0.00180306], dtype=float32)

In [21]:
evaluate("Stupid motherfucker, get lost")

array([0.9999726], dtype=float32)

In [22]:
train_dataset["comment_text"][12]

'Hey... what is it..\n@ | talk .\nWhat is it... an exclusive group of some WP TALIBANS...who are good at destroying, self-appointed purist who GANG UP any one who asks them questions abt their ANTI-SOCIAL and DESTRUCTIVE (non)-contribution at WP?\n\nAsk Sityush to clean up his behavior than issue me nonsensical warnings...'

In [23]:
evaluate(train_dataset["comment_text"][12])

array([0.9993055], dtype=float32)